In [13]:
# !pip install transformers

In [1]:
import optuna
from src.subset_dataloader import sub_create_dataloader
from src.model import Model
from src.utils.torch_utils import model_info, check_runtime
from src.trainer import TorchTrainer, count_model_params
from typing import Any, Dict, List, Tuple
import argparse
import os
from datetime import datetime
from typing import Any, Dict, Tuple, Union

import torch
import torch.nn as nn
import torch.optim as optim
import yaml

from src.dataloader import create_dataloader
from src.loss import CustomCriterion
from src.model import Model
from src.trainer import TorchTrainer
from src.utils.common import get_label_counts, read_yaml
from src.utils.torch_utils import check_runtime, model_info

import random
import numpy as np

from transformers import AdamW, get_linear_schedule_with_warmup

In [2]:
def set_seed(seed: int):
    """
    Helper function for reproducible behavior to set the seed in ``random``, ``numpy``, ``torch`` and/or ``tf`` (if
    installed).

    Args:
        seed (:obj:`int`): The seed to set.
    """
    random.seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        # ^^ safe to call this function even if cuda is not available
set_seed(42)

In [3]:
if not torch.cuda.is_available():
    device = torch.device("cpu")
else:
    device = torch.device(f"cuda:0")

In [4]:
DATA_PATH = '../data'
RESULT_MODEL_PATH = './final_best.pt'

In [17]:
def search_hyperparam(trial: optuna.trial.Trial) -> Dict[str, Any]:
    """Search hyperparam from user-specified search space."""
    epochs = 50
    img_size = 96 # tuning할 때만, 96을 활용
    n_select = trial.suggest_int("n_select", low=0, high=6, step=2)
    batch_size = trial.suggest_int("batch_size", low=32, high=128, step=32)
    # lr = trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True)
    lr = 0.01
    optim = trial.suggest_categorical('optimizer', ['MomentumSGD', 'AdamW'])
    scheduler = trial.suggest_categorical('scheduler', ['linear', 'cycle'])

    return {
        "EPOCHS": epochs,
        "IMG_SIZE": img_size,
        "n_select": n_select,
        "BATCH_SIZE": batch_size,
        "INIT_LR": lr,
        "optimizer": optim,
        "scheduler": scheduler
    }

In [18]:
model_config = read_yaml(cfg='/opt/ml/code/search_model/model_23.yaml')

In [20]:
def objective(trial: optuna.trial.Trial, device, fp16) -> Tuple[float, int, float]:
    """Optuna objective.
    Args:
        trial
    Returns:
        float: score1(e.g. accuracy)
        int: score2(e.g. params)
    """
    model = Model(model_config, verbose=True)
    model.to(device)
    model.model.to(device)

    hyperparams = search_hyperparam(trial)
    # check ./data_configs/data.yaml for config information
    data_config: Dict[str, Any] = {}
    data_config["DATA_PATH"] = DATA_PATH
    data_config["DATASET"] = "TACO"
    data_config["AUG_TRAIN"] = "randaugment_train" # default: randaugment_train
    data_config["AUG_TEST"] = "simple_augment_test"
    data_config["AUG_TRAIN_PARAMS"] = {
        "n_select": hyperparams["n_select"],
    }
    data_config['AUG_TRAIN_PARAMS'] = None
    data_config["AUG_TEST_PARAMS"] = None
    data_config["BATCH_SIZE"] = hyperparams["BATCH_SIZE"]
    data_config["VAL_RATIO"] = 0.8
    data_config["IMG_SIZE"] = hyperparams["IMG_SIZE"]
    data_config["INIT_LR"] = hyperparams["INIT_LR"]
    data_config["EPOCHS"] = hyperparams['EPOCHS']
    data_config["optimizer"] = hyperparams["optimizer"]
    data_config["scheduler"] = hyperparams["scheduler"]
    """이부분이 config를 저장하는 부분입니다. 위의 lr,fp16,epochs는 원래 함수에는 없지만
    config를 바로 사용할 수 있게 추가했습니다."""
    
    k = 1
    file_name = f"search_params_without_lr/model_{k}.yaml"
    while os.path.exists(file_name):
        print(k)
        k += 1
        file_name = f"search_params_without_lr/model_{k}.yaml"
    print(model_config)
    "model config와 data config를 저장"
    with open(f"search_params_without_lr/model_{k}.yaml", "w") as outfile:
        yaml.dump(model_config, outfile)
    with open(f"search_params_without_lr/data_{k}.yaml", "w") as outfile:
        yaml.dump(data_config, outfile)

    mean_time = check_runtime(
        model.model,
        [model_config["input_channel"]] + model_config["INPUT_SIZE"],
        device,
    )
    model_info(model, verbose=True)
    train_loader, val_loader, test_loader = sub_create_dataloader(data_config)

    # criterion = nn.CrossEntropyLoss()
    if data_config["optimizer"] == 'MomentumSGD':
        optimizer = torch.optim.SGD(model.parameters(), lr=data_config['INIT_LR'], momentum=0.9)
    elif data_config["optimizer"] == 'AdamW':
        optimizer = AdamW(model.parameters(), lr=data_config['INIT_LR'], eps=1e-8)

    if data_config["scheduler"] == 'linear':
        t_total = len(train_loader) * data_config["EPOCHS"]
        warmup_steps = int(t_total * 0.1)
        scheduler = get_linear_schedule_with_warmup(
            optimizer, num_warmup_steps=warmup_steps, num_training_steps=t_total
        )  
    elif data_config['scheduler'] == 'cycle':
        scheduler = torch.optim.lr_scheduler.OneCycleLR(
            optimizer,
            max_lr=data_config["INIT_LR"],
            steps_per_epoch=len(train_loader),
            epochs=hyperparams["EPOCHS"],
            pct_start=0.05,
        )
    # optimizer = torch.optim.SGD(model.parameters(), lr=data_config["INIT_LR"])
    # scheduler = torch.optim.lr_scheduler.OneCycleLR(
    #     optimizer,
    #     max_lr=data_config["INIT_LR"],
    #     steps_per_epoch=len(train_loader),
    #     epochs=hyperparams["EPOCHS"],
    #     pct_start=0.05,
    # )

    criterion = CustomCriterion(
        samples_per_cls=get_label_counts(data_config["DATA_PATH"])
        if data_config["DATASET"] == "TACO"
        else None,
        device=device,
    )
    # Amp loss scaler
    scaler = (
        torch.cuda.amp.GradScaler() if fp16 and device != torch.device("cpu") else None
    )
    # scaler = torch.cuda.amp.GradScaler() if fp16 and device != torch.device("cpu") else None

    trainer = TorchTrainer(
        model,
        criterion,
        optimizer,
        scheduler,
        device=device,
        verbose=1,
        model_path=RESULT_MODEL_PATH,
        scaler=scaler,
    )
    trainer.train(train_loader, hyperparams["EPOCHS"], val_dataloader=val_loader)
    loss, f1_score, acc_percent = trainer.test(model, test_dataloader=val_loader)
    params_nums = count_model_params(model)

    model_info(model, verbose=True)
    return f1_score, params_nums, mean_time

In [21]:
def get_best_trial_with_condition(optuna_study: optuna.study.Study) -> Dict[str, Any]:
    """Get best trial that satisfies the minimum condition(e.g. accuracy > 0.8).
    Args:
        study : Optuna study object to get trial.
    Returns:
        best_trial : Best trial that satisfies condition.
    """
    df = optuna_study.trials_dataframe().rename(
        columns={
            "values_0": "acc_percent",
            "values_1": "params_nums",
            "values_2": "mean_time",
        }
    )
    ## minimum condition : accuracy >= threshold
    threshold = 0.7
    minimum_cond = df.acc_percent >= threshold

    if minimum_cond.any():
        df_min_cond = df.loc[minimum_cond]
        ## get the best trial idx with lowest parameter numbers
        best_idx = df_min_cond.loc[
            df_min_cond.params_nums == df_min_cond.params_nums.min()
        ].acc_percent.idxmax()

        best_trial_ = optuna_study.trials[best_idx]
        print("Best trial which satisfies the condition")
        print(df.loc[best_idx])
    else:
        print("No trials satisfies minimum condition")
        best_trial_ = None

    return best_trial_

In [23]:
def tune(gpu_id, storage: str = None, fp16: bool = False):
    if not torch.cuda.is_available():
        device = torch.device("cpu")
    elif 0 <= gpu_id < torch.cuda.device_count():
        device = torch.device(f"cuda:{gpu_id}")
    sampler = optuna.samplers.MOTPESampler()
    if storage is not None:
        rdb_storage = optuna.storages.RDBStorage(url=storage)
    else:
        rdb_storage = None
    study = optuna.create_study(
        directions=["maximize", "minimize", "minimize"],
        study_name="automl",
        sampler=sampler,
        storage=rdb_storage,
        load_if_exists=True,
    )
    study.optimize(lambda trial: objective(trial, device, fp16), n_trials=20)
    pruned_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED]
    complete_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]

    print("Study statistics: ")
    print("  Number of finished trials: ", len(study.trials))
    print("  Number of pruned trials: ", len(pruned_trials))
    print("  Number of complete trials: ", len(complete_trials))

    print("Best trials:")
    best_trials = study.best_trials

    ## trials that satisfies Pareto Fronts
    for tr in best_trials:
        print(f"  value1:{tr.values[0]}, value2:{tr.values[1]}")
        for key, value in tr.params.items():
            print(f"    {key}:{value}")

    best_trial = get_best_trial_with_condition(study)
    print(best_trial)
    df = study.trials_dataframe(attrs=("number", "value", "params", "state"))
    df.to_csv("search_results.csv", index=False)

In [24]:
parser = argparse.ArgumentParser(description="Optuna tuner.")
parser.add_argument("--gpu", default=0, type=int, help="GPU id to use")
parser.add_argument("--storage", default="", type=str, help="Optuna database storage path.")
parser.add_argument("--fp16", default=False, type=bool, help="train to fp16")
args = parser.parse_args([])  
tune(args.gpu, storage=args.storage if args.storage != "" else None, fp16=args.fp16)

<ipython-input-23-7093b73e0f9f>:6: ExperimentalWarning: MOTPESampler is experimental (supported from v2.4.0). The interface can change in the future.
  sampler = optuna.samplers.MOTPESampler()
[I 2021-11-29 05:07:49,773] A new study created in memory with name: automl


idx |   n |     params |          module |            arguments |   in_channel |   out_channel
----------------------------------------------------------------------------------------------
  0 |   1 |        688 | InvertedResidualv3 | [3, 5.4, 48, 0, 1, 2] |            3           24
  1 |   1 |        928 |          DWConv | [54, 3, 1, None, 'Hardswish'] |           24           32
  2 |   1 |        960 |          DWConv | [93, 3, 1, None, 'Hardswish'] |           32           48
  3 |   1 |     13,888 |            Conv | [63, 3, 2, None, 1, 'Hardswish'] |           48           32
  4 |   1 |      2,736 |          DWConv | [144, 3, 1, None, 'Hardswish'] |           32           72
  5 |   1 |     38,408 | InvertedResidualv3 | [5, 4.5, 32, 0, 1, 2] |           72           16
  6 |   1 |      9,344 |            Conv | [125, 3, 2, None, 1, 'ReLU'] |           16           64
  7 |   1 |     13,632 | InvertedResidualv3 | [5, 1.5, 88, 0, 0, 2] |           64           48
  8 |   1 |   

Train: [001] Loss: 1.975, Acc: 21.14% F1(macro): 0.18: 100%|██████████| 36/36 [00:18<00:00,  1.95it/s]
 Val:       Loss: 1.935, Acc: 6.97% F1(macro): 0.04: 100%|██████████| 41/41 [00:29<00:00,  1.39it/s]


Model saved. Current best test f1: 0.044


Train: [002] Loss: 1.718, Acc: 33.22% F1(macro): 0.22: 100%|██████████| 36/36 [00:17<00:00,  2.05it/s]
 Val:       Loss: 1.807, Acc: 37.56% F1(macro): 0.21: 100%|██████████| 41/41 [00:29<00:00,  1.40it/s]


...

Train: [018] Loss: 1.424, Acc: 43.38% F1(macro): 0.27: 100%|██████████| 36/36 [00:15<00:00,  2.33it/s]
 Val:       Loss: 1.343, Acc: 47.50% F1(macro): 0.32: 100%|██████████| 41/41 [00:29<00:00,  1.41it/s]


Model saved. Current best test f1: 0.322


Train: [019] Loss: 1.426, Acc: 42.93% F1(macro): 0.27: 100%|██████████| 36/36 [00:18<00:00,  1.90it/s]
 Val:       Loss: 1.459, Acc: 42.06% F1(macro): 0.27: 100%|██████████| 41/41 [00:25<00:00,  1.58it/s]
Train: [020] Loss: 1.427, Acc: 43.36% F1(macro): 0.27: 100%|██████████| 36/36 [00:16<00:00,  2.20it/s]
 Val:       Loss: 1.351, Acc: 47.23% F1(macro): 0.34: 100%|██████████| 41/41 [00:27<00:00,  1.48it/s]


Model saved. Current best test f1: 0.343


Train: [021] Loss: 1.409, Acc: 44.01% F1(macro): 0.28: 100%|██████████| 36/36 [00:17<00:00,  2.04it/s]
 Val:       Loss: 1.365, Acc: 46.36% F1(macro): 0.30: 100%|██████████| 41/41 [00:25<00:00,  1.59it/s]
Train: [022] Loss: 1.412, Acc: 44.23% F1(macro): 0.29: 100%|██████████| 36/36 [00:16<00:00,  2.22it/s]
 Val:       Loss: 1.389, Acc: 46.17% F1(macro): 0.27: 100%|██████████| 41/41 [00:27<00:00,  1.50it/s]
Train: [023] Loss: 1.417, Acc: 43.27% F1(macro): 0.27: 100%|██████████| 36/36 [00:17<00:00,  2.03it/s]
 Val:       Loss: 1.343, Acc: 47.32% F1(macro): 0.32: 100%|██████████| 41/41 [00:26<00:00,  1.55it/s]
Train: [024] Loss: 1.397, Acc: 45.03% F1(macro): 0.28: 100%|██████████| 36/36 [00:16<00:00,  2.18it/s]
 Val:       Loss: 1.371, Acc: 46.73% F1(macro): 0.29: 100%|██████████| 41/41 [00:26<00:00,  1.55it/s]
Train: [025] Loss: 1.380, Acc: 45.53% F1(macro): 0.30: 100%|██████████| 36/36 [00:17<00:00,  2.06it/s]
 Val:       Loss: 1.361, Acc: 46.67% F1(macro): 0.28: 100%|██████████| 41/41 